<a href="https://colab.research.google.com/github/finiteautomata/hatespeech-classification/blob/main/notebooks/Hatespeech_Colab_TPU_Single.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrenamiento de modelos de Hate Speech usando TPUs

En esta notebook haremos el entrenamiento de un modelo "pesado" de Hate Speech. La idea es usar las TPU para poder entrenar un BERT que toma ~ 10 hs en dos GPU

Ahora, a montar Drive

In [1]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --version nightly
!python pytorch-xla-env-setup.py --version 1.8.1

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  35776      0 --:--:-- --:--:-- --:--:-- 35776
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-1.8.1 ...
     |████████████████████████████████| 61kB 2.7MB/s 
Uninstalling torch-1.8.1+cu101:
ERROR: earthengine-api 0.1.269 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 1.8.0 which is incompatible.
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
Done updating TPU runtime
  Successfully uninstalled torch-1.8.1+cu101
Uninstalling torchvision-0.9.1+cu101:
  Successfully uninstalled torchvision-0.9.1+cu101
Copying gs://tpu-pytorch/wheels/torch-1.8.1-cp37-cp37m-linux_x86_64.whl...
\ [1 files][126.5 MiB/126.

In [2]:
from google.colab import drive

drive.mount("/content/drive/")

Mounted at /content/drive/


Al toque perrito

In [3]:
repo_dir = "/content/drive/Shareddrives/HateSpeech/hatespeech-classification"
!cd $repo_dir && git pull
!cd $repo_dir && pip install -e . 
#!pip install absl-py

Already up to date.
Obtaining file:///content/drive/Shareddrives/HateSpeech/hatespeech-classification
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 194kB 5.0MB/s 
     |████████████████████████████████| 2.1MB 20.3MB/s 
     |████████████████████████████████| 245kB 40.8MB/s 
     |████████████████████████████████| 122kB 16.6MB/s 
     |████████████████████████████████| 901kB 31.5MB/s 
     |████████████████████████████████| 3.3MB 39.5MB/s 
     |████████████████████████████████| 133kB 13.1MB/s 
  Created wheel for pandarallel: filename=pandarallel-1.5.2-cp37-none-any.whl size=18399 sha256=f0797f4850b716b1db0cf6bbe0c58327952cafdbe8f148c0e1f771fe12c5b174
  Stored in directory: /root/.cache/pip/wheels/40/80/6d/d50fb72a8ce6a923fb10390fec9eaaa40b02d07a7ec05c9c05
Successfully built pandarallel
ERROR: pysentimiento 0.2.0 has requirement transformers==4.6.1, but you'll have tr

## **NOTA IMPORTANTE**

Reiniciar notebook después de correr la anterior celda. Por algún motivo extraño, no reconoce el paquete que acabamos de instalar localmente

In [1]:
from hatedetection.metrics import compute_extended_category_metrics
from hatedetection.preprocessing import special_tokens
from hatedetection.training import load_hatespeech_model_and_tokenizer, tokenize
from transformers import (
    BertTokenizerFast, TrainingArguments, Trainer,
    AutoModelForSequenceClassification, AutoTokenizer
)

def train_hatespeech_classifier(
    model, train_dataset, dev_dataset, seed,
    max_length=None, epochs=10, warmup_proportion=0.1, 
    ):
    per_device_train_batch_size = 16
    num_cores = 8
    batch_size = num_cores * per_device_train_batch_size

    total_steps = (epochs * len(train_dataset)) // batch_size
    warmup_steps = int(warmup_proportion * total_steps)
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=epochs,
        warmup_steps=warmup_steps,
        evaluation_strategy="epoch",
        per_device_train_batch_size=per_device_train_batch_size,
        weight_decay=0.01,
        logging_dir='./logs',
        eval_accumulation_steps=1,
        seed=seed,
        #load_best_model_at_end=True,
        #metric_for_best_model="f1",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=lambda pred: compute_extended_category_metrics(dev_dataset, pred),
        train_dataset=train_dataset,
        eval_dataset=dev_dataset,
    )

    trainer.place_model_on_device = False

    trainer.train()

    return trainer


In [2]:
import torch

from hatedetection import BertForSequenceMultiClassification, load_datasets, extended_hate_categories
from hatedetection.training import tokenize, lengths
from hatedetection.preprocessing import special_tokens
from hatedetection.metrics import compute_extended_category_metrics


def load_model_and_tokenizer(model_name, context, max_length=None, class_weight=None):
    """
    Load model and tokenizer
    """

    if not max_length:
        max_length = lengths[context]

    model = BertForSequenceMultiClassification.from_pretrained(
        model_name, return_dict=True, num_labels=len(extended_hate_categories),
        pos_weight=class_weight,
    )

    model.train()

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.model_max_length = max_length

    vocab = tokenizer.get_vocab()
    new_tokens_to_add = [tok for tok in special_tokens if tok not in tokenizer.get_vocab()]

    if new_tokens_to_add:
        """
        TODO: Perdoname Wilkinson, te he fallado

        Hay una interfaz diferente acá, no entiendo bien por qué
        """
        if type(tokenizer) is BertTokenizerFast:
            tokenizer.add_special_tokens({'additional_special_tokens': new_tokens_to_add})
        else:
            tokenizer.add_special_tokens(new_tokens_to_add)
        model.resize_token_embeddings(len(tokenizer))

    return model, tokenizer

model_name = 'dccuchile/bert-base-spanish-wwm-uncased'
context = 'title+body'


add_body = True if 'body' in context else False
train_dataset, dev_dataset, test_dataset = load_datasets(add_body=add_body)



labels = torch.Tensor([train_dataset[c] for c in extended_hate_categories]).T

class_weight = (1 / (2 * labels.mean(0)))


"""
Todo esto es para evitar que la memoria salte por los aires por usar 'fork' en XLA
Ver: https://pytorch.org/xla/release/1.8/index.html#torch_xla.distributed.xla_multiprocessing.MpModelWrapper

"""
print("Cargando modelo")

model, tokenizer = load_model_and_tokenizer(model_name, context, class_weight=class_weight)



"""
TODO: ver si podemos salvarlo primero

https://huggingface.co/docs/datasets/processing.html#saving-a-processed-dataset-on-disk-and-reload-it
"""


tokenize_fun = lambda batch: tokenize(tokenizer, batch, context=context)
train_dataset = train_dataset.map(tokenize_fun, batched=True, batch_size=64)
dev_dataset = dev_dataset.map(tokenize_fun, batched=True, batch_size=64)
test_dataset = test_dataset.map(tokenize_fun, batched=True, batch_size=64)


def format_dataset(dataset):
    def get_category_labels(examples):
        return {'labels': torch.Tensor([examples[cat] for cat in extended_hate_categories])}
    dataset = dataset.map(get_category_labels)
    dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
    return dataset
    
train_dataset = format_dataset(train_dataset)
dev_dataset = format_dataset(dev_dataset)
test_dataset = format_dataset(test_dataset)

Cargando modelo


Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForSequenceMultiClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceMultiClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceMultiClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceMultiClassification were not initialized from the model

Los salvamos a disco. Esto es porque tarda un huevo si lo hacemos 8 veces.

In [3]:
train_path = "datasets/train/"
dev_path = "datasets/dev/"
test_path = "datasets/test"

train_dataset.save_to_disk(train_path)
dev_dataset.save_to_disk(dev_path)
test_dataset.save_to_disk(test_path)

Sanity Check

In [4]:

tokenizer.decode(test_dataset[0]["input_ids"])

'[CLS] [USER] ojalá se funda y cague de hambre [SEP] mora godoy cierra su escuela de tango y remata su vestuario para " poder seguir adelante " - puede marcar con justeza la fecha de inicio de la crisis : hace 10 meses que no brinda clases. en cambio, no logra avizorar el día que concluirá : " no sabemos cuándo empezamos, cuando volvemos a trabajar. ¿ ideas? ¡ ( tenemos ) miles! podemos hacer un show gratis al aire libre ". es por eso que, ante la falta de un horizonte, mora godoy decidió cerrar su escuela de danzas y rematar gran parte del vestuario para poder continuar adelante. " hace 20 años que estoy acá, pero llevo diez meses sin trabajar - le explicó la artista a guillermo andino en el programa informados de todo, por américa -. este es mi lugar de ensayo, de creación. y es muy duro. para poder seguir adelante me tengo que desprender del lugar y rematar gran parte del vestuario. que muchos bailarines y bailarines lo van a aprovechar porque el precio es muy bajo ". por caso, traj

In [5]:
import os
import torch
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
from datasets import load_from_disk
import sys


"""
Todo esto es para evitar que la memoria salte por los aires por usar 'fork' en XLA
Ver: https://pytorch.org/xla/release/1.8/index.html#torch_xla.distributed.xla_multiprocessing.MpModelWrapper

"""

WRAPPED_MODEL = xmp.MpModelWrapper(model)


def train_model(
    model, tokenizer, output_path, seed=None,
    batch_size=8, eval_batch_size=8, max_length=None, slice=None,
    epochs=10, warmup_proportion=0.1,
    ):
    """
    """ 
    print("*"*80)
    print("Training hate speech classifier")


    if not seed:
        seed = np.random.randint(0, 20_000)
    print("Loading datasets... ", end="")

    train_dataset = load_from_disk(train_path)
    dev_dataset = load_from_disk(dev_path)

    if slice:
        print("Slicing")
        train_dataset = train_dataset.select(range(slice))
        dev_dataset = dev_dataset.select(range(slice))
    print("Done")


    trainer = train_hatespeech_classifier(
        model, train_dataset=train_dataset, dev_dataset=dev_dataset,
        epochs=epochs, warmup_proportion=warmup_proportion, seed=seed,
    )
    print("Terminamos de entrenar perrito!")
    print("\n"*3, "Saving...")
    trainer.save_model(output_path)
    tokenizer.save_pretrained(output_path)

    print(f"Models saved at {output_path}")



In [7]:
import os
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp

for i in range(1, 16):
    output_path = f"/content/drive/Shareddrives/HateSpeech/models/bert-single-title-body_{i}/"

    if os.path.isdir(output_path):
        print(f"{output_path} already trained -- skipping ")
        continue
    seed = 2020 + i 
    
    def _mp_fn(index):
        device = xm.xla_device()
        xm.set_rng_state(seed, device=device)

        model = WRAPPED_MODEL.to(device)


        train_model(
            model, tokenizer, epochs=10, output_path=output_path,
            batch_size=8, seed=seed, 
            #slice=2048,
        )

    xmp.spawn(_mp_fn, start_method="fork")

/content/drive/Shareddrives/HateSpeech/models/bert-single-title-body_1/ already trained -- skipping 
/content/drive/Shareddrives/HateSpeech/models/bert-single-title-body_2/ already trained -- skipping 
/content/drive/Shareddrives/HateSpeech/models/bert-single-title-body_3/ already trained -- skipping 
/content/drive/Shareddrives/HateSpeech/models/bert-single-title-body_4/ already trained -- skipping 
/content/drive/Shareddrives/HateSpeech/models/bert-single-title-body_5/ already trained -- skipping 
/content/drive/Shareddrives/HateSpeech/models/bert-single-title-body_6/ already trained -- skipping 
/content/drive/Shareddrives/HateSpeech/models/bert-single-title-body_7/ already trained -- skipping 
/content/drive/Shareddrives/HateSpeech/models/bert-single-title-body_8/ already trained -- skipping 
/content/drive/Shareddrives/HateSpeech/models/bert-single-title-body_9/ already trained -- skipping 
/content/drive/Shareddrives/HateSpeech/models/bert-single-title-body_10/ already trained --

Epoch,Training Loss,Validation Loss,Calls F1,Women F1,Lgbti F1,Racism F1,Class F1,Politics F1,Disabled F1,Appearance F1,Criminal F1,Mean F1,Mean Precision,Mean Recall,Hate Precision,Hate Recall,Hate F1,Runtime,Samples Per Second
1,No log,0.458599,0.419847,0.360802,0.459770,0.475806,0.376518,0.510046,0.290503,0.568807,0.638484,0.455620,0.350208,0.700605,0.505806,0.785148,0.615254,85.914700,105.989000
2,0.551600,0.351591,0.479495,0.399497,0.523148,0.529070,0.466830,0.555730,0.559585,0.636943,0.641350,0.532406,0.409575,0.794492,0.489396,0.865177,0.625163,66.503000,136.926000
3,0.551600,0.554586,0.684318,0.497087,0.594595,0.713217,0.511041,0.637405,0.567010,0.702509,0.740741,0.627547,0.583446,0.690500,0.685319,0.750541,0.716449,66.842500,136.231000
4,0.237300,0.573151,0.679035,0.525050,0.657244,0.700210,0.528190,0.662745,0.666667,0.714286,0.727915,0.651260,0.593433,0.725516,0.667275,0.790916,0.723854,66.590600,136.746000
5,0.237300,0.705911,0.718563,0.528302,0.637993,0.714286,0.566553,0.689655,0.601093,0.727273,0.737564,0.657920,0.618399,0.706094,0.695003,0.772170,0.731557,66.982800,135.945000
6,0.099600,0.765833,0.722656,0.539095,0.648276,0.705085,0.539007,0.654832,0.627660,0.761165,0.772388,0.663352,0.628957,0.705743,0.692407,0.775775,0.731724,66.806800,136.303000
7,0.099600,0.873904,0.720792,0.581281,0.626335,0.714116,0.543624,0.723404,0.621469,0.766355,0.741818,0.671022,0.653625,0.693171,0.726396,0.759913,0.742777,68.199300,133.520000
8,0.038100,0.945778,0.700219,0.554054,0.649635,0.707617,0.530909,0.735892,0.644068,0.763916,0.743494,0.669978,0.672191,0.671672,0.724597,0.745494,0.734897,67.997100,133.917000
9,0.012500,0.969604,0.701754,0.572093,0.647482,0.703614,0.550000,0.728070,0.604651,0.764259,0.759398,0.670147,0.670395,0.673604,0.726829,0.751983,0.739192,69.073600,131.830000
10,0.012500,1.002747,0.702820,0.573460,0.639706,0.704019,0.553506,0.725664,0.632184,0.759924,0.776923,0.674245,0.679612,0.671382,0.737705,0.746215,0.741935,69.327100,131.348000


********************************************************************************
Training hate speech classifier
Loading datasets... Done


Epoch,Training Loss,Validation Loss,Calls F1,Women F1,Lgbti F1,Racism F1,Class F1,Politics F1,Disabled F1,Appearance F1,Criminal F1,Mean F1,Mean Precision,Mean Recall,Hate Precision,Hate Recall,Hate F1,Runtime,Samples Per Second
1,No log,0.458599,0.419847,0.360802,0.459770,0.475806,0.376518,0.510046,0.290503,0.568807,0.638484,0.455620,0.350208,0.700605,0.505806,0.785148,0.615254,86.132800,105.720000
2,0.564600,0.351591,0.479495,0.399497,0.523148,0.529070,0.466830,0.555730,0.559585,0.636943,0.641350,0.532406,0.409575,0.794492,0.489396,0.865177,0.625163,66.499600,136.933000
3,0.564600,0.554586,0.684318,0.497087,0.594595,0.713217,0.511041,0.637405,0.567010,0.702509,0.740741,0.627547,0.583446,0.690500,0.685319,0.750541,0.716449,66.889400,136.135000
4,0.195800,0.573151,0.679035,0.525050,0.657244,0.700210,0.528190,0.662745,0.666667,0.714286,0.727915,0.651260,0.593433,0.725516,0.667275,0.790916,0.723854,66.582500,136.763000
5,0.195800,0.705911,0.718563,0.528302,0.637993,0.714286,0.566553,0.689655,0.601093,0.727273,0.737564,0.657920,0.618399,0.706094,0.695003,0.772170,0.731557,66.919800,136.073000
6,0.099700,0.765833,0.722656,0.539095,0.648276,0.705085,0.539007,0.654832,0.627660,0.761165,0.772388,0.663352,0.628957,0.705743,0.692407,0.775775,0.731724,66.899300,136.115000
7,0.099700,0.873904,0.720792,0.581281,0.626335,0.714116,0.543624,0.723404,0.621469,0.766355,0.741818,0.671022,0.653625,0.693171,0.726396,0.759913,0.742777,68.206000,133.507000
8,0.044400,0.945778,0.700219,0.554054,0.649635,0.707617,0.530909,0.735892,0.644068,0.763916,0.743494,0.669978,0.672191,0.671672,0.724597,0.745494,0.734897,68.017900,133.877000
9,0.011900,0.969604,0.701754,0.572093,0.647482,0.703614,0.550000,0.728070,0.604651,0.764259,0.759398,0.670147,0.670395,0.673604,0.726829,0.751983,0.739192,69.028300,131.917000
10,0.011900,1.002747,0.702820,0.573460,0.639706,0.704019,0.553506,0.725664,0.632184,0.759924,0.776923,0.674245,0.679612,0.671382,0.737705,0.746215,0.741935,69.405900,131.199000


********************************************************************************
Training hate speech classifier
Loading datasets... Done


...

********************************************************************************
Training hate speech classifier
Loading datasets... Done


Epoch,Training Loss,Validation Loss,Calls F1,Women F1,Lgbti F1,Racism F1,Class F1,Politics F1,Disabled F1,Appearance F1,Criminal F1,Mean F1,Mean Precision,Mean Recall,Hate Precision,Hate Recall,Hate F1,Runtime,Samples Per Second
1,No log,0.458599,0.419847,0.360802,0.459770,0.475806,0.376518,0.510046,0.290503,0.568807,0.638484,0.455620,0.350208,0.700605,0.505806,0.785148,0.615254,86.126100,105.729000
2,0.585200,0.351591,0.479495,0.399497,0.523148,0.529070,0.466830,0.555730,0.559585,0.636943,0.641350,0.532406,0.409575,0.794492,0.489396,0.865177,0.625163,66.567700,136.793000
3,0.585200,0.554586,0.684318,0.497087,0.594595,0.713217,0.511041,0.637405,0.567010,0.702509,0.740741,0.627547,0.583446,0.690500,0.685319,0.750541,0.716449,66.928500,136.056000
4,0.204800,0.573151,0.679035,0.525050,0.657244,0.700210,0.528190,0.662745,0.666667,0.714286,0.727915,0.651260,0.593433,0.725516,0.667275,0.790916,0.723854,66.666900,136.590000
5,0.204800,0.705911,0.718563,0.528302,0.637993,0.714286,0.566553,0.689655,0.601093,0.727273,0.737564,0.657920,0.618399,0.706094,0.695003,0.772170,0.731557,66.999500,135.912000
6,0.091300,0.765833,0.722656,0.539095,0.648276,0.705085,0.539007,0.654832,0.627660,0.761165,0.772388,0.663352,0.628957,0.705743,0.692407,0.775775,0.731724,66.880200,136.154000
7,0.091300,0.873904,0.720792,0.581281,0.626335,0.714116,0.543624,0.723404,0.621469,0.766355,0.741818,0.671022,0.653625,0.693171,0.726396,0.759913,0.742777,68.168200,133.581000
8,0.042200,0.945778,0.700219,0.554054,0.649635,0.707617,0.530909,0.735892,0.644068,0.763916,0.743494,0.669978,0.672191,0.671672,0.724597,0.745494,0.734897,67.976300,133.958000
9,0.014700,0.969604,0.701754,0.572093,0.647482,0.703614,0.550000,0.728070,0.604651,0.764259,0.759398,0.670147,0.670395,0.673604,0.726829,0.751983,0.739192,69.111000,131.759000
10,0.014700,1.002747,0.702820,0.573460,0.639706,0.704019,0.553506,0.725664,0.632184,0.759924,0.776923,0.674245,0.679612,0.671382,0.737705,0.746215,0.741935,69.325400,131.352000


********************************************************************************
Training hate speech classifier
Loading datasets... Done


Epoch,Training Loss,Validation Loss,Calls F1,Women F1,Lgbti F1,Racism F1,Class F1,Politics F1,Disabled F1,Appearance F1,Criminal F1,Mean F1,Mean Precision,Mean Recall,Hate Precision,Hate Recall,Hate F1,Runtime,Samples Per Second
1,No log,0.458599,0.419847,0.360802,0.459770,0.475806,0.376518,0.510046,0.290503,0.568807,0.638484,0.455620,0.350208,0.700605,0.505806,0.785148,0.615254,85.911400,105.993000
2,0.579000,0.351591,0.479495,0.399497,0.523148,0.529070,0.466830,0.555730,0.559585,0.636943,0.641350,0.532406,0.409575,0.794492,0.489396,0.865177,0.625163,66.499100,136.934000
3,0.579000,0.554586,0.684318,0.497087,0.594595,0.713217,0.511041,0.637405,0.567010,0.702509,0.740741,0.627547,0.583446,0.690500,0.685319,0.750541,0.716449,66.947700,136.017000
4,0.210000,0.573151,0.679035,0.525050,0.657244,0.700210,0.528190,0.662745,0.666667,0.714286,0.727915,0.651260,0.593433,0.725516,0.667275,0.790916,0.723854,66.599600,136.728000
5,0.210000,0.705911,0.718563,0.528302,0.637993,0.714286,0.566553,0.689655,0.601093,0.727273,0.737564,0.657920,0.618399,0.706094,0.695003,0.772170,0.731557,66.919400,136.074000
6,0.111500,0.765833,0.722656,0.539095,0.648276,0.705085,0.539007,0.654832,0.627660,0.761165,0.772388,0.663352,0.628957,0.705743,0.692407,0.775775,0.731724,66.919000,136.075000
7,0.111500,0.873904,0.720792,0.581281,0.626335,0.714116,0.543624,0.723404,0.621469,0.766355,0.741818,0.671022,0.653625,0.693171,0.726396,0.759913,0.742777,68.199900,133.519000
8,0.045100,0.945778,0.700219,0.554054,0.649635,0.707617,0.530909,0.735892,0.644068,0.763916,0.743494,0.669978,0.672191,0.671672,0.724597,0.745494,0.734897,67.994000,133.924000
9,0.014600,0.969604,0.701754,0.572093,0.647482,0.703614,0.550000,0.728070,0.604651,0.764259,0.759398,0.670147,0.670395,0.673604,0.726829,0.751983,0.739192,69.062600,131.851000
10,0.014600,1.002747,0.702820,0.573460,0.639706,0.704019,0.553506,0.725664,0.632184,0.759924,0.776923,0.674245,0.679612,0.671382,0.737705,0.746215,0.741935,69.339800,131.324000


Terminamos de entrenar perrito!



 Saving...
Terminamos de entrenar perrito!
Terminamos de entrenar perrito!






 Saving...
Terminamos de entrenar perrito!
 Saving...



 Saving...
Terminamos de entrenar perrito!
Terminamos de entrenar perrito!



Terminamos de entrenar perrito!



 Saving...



 Saving...
 Saving...
Terminamos de entrenar perrito!



 Saving...
Models saved at /content/drive/Shareddrives/HateSpeech/models/bert-single-title-body_15/
Models saved at /content/drive/Shareddrives/HateSpeech/models/bert-single-title-body_15/
Models saved at /content/drive/Shareddrives/HateSpeech/models/bert-single-title-body_15/
Models saved at /content/drive/Shareddrives/HateSpeech/models/bert-single-title-body_15/
Models saved at /content/drive/Shareddrives/HateSpeech/models/bert-single-title-body_15/
Models saved at /content/drive/Shareddrives/HateSpeech/models/bert-single-title-body_15/
Models saved at /content/drive/Shareddrives/HateSpeech/models/bert-single-title-body_15/
Models sa

In [ ]:

!sudo dmesg -T

[Fri Jun 11 13:35:44 2021] Linux version 5.4.104+ (builder@81cc40d87d7b) (Chromium OS 12.0_pre408248_p20201125-r7 clang version 12.0.0 (/var/tmp/portage/sys-devel/llvm-12.0_pre408248_p20201125-r7/work/llvm-12.0_pre408248_p20201125/clang f402e682d0ef5598eeffc9a21a691b03e602ff58)) #1 SMP Sat Jun 5 09:50:34 PDT 2021
[Fri Jun 11 13:35:44 2021] Command line: BOOT_IMAGE=/syslinux/vmlinuz.A init=/usr/lib/systemd/systemd boot=local rootwait ro noresume noswap loglevel=7 noinitrd console=ttyS0 security=apparmor virtio_net.napi_tx=1 systemd.unified_cgroup_hierarchy=false systemd.legacy_systemd_cgroup_controller=false csm.disabled=1 loadpin.exclude=kernel-module modules-load=loadpin_trigger module.sig_enforce=1 console=tty1 dm_verity.error_behavior=3 dm_verity.max_bios=-1 dm_verity.dev_wait=1 i915.modeset=1 cros_efi root=/dev/dm-0 "dm=1 vroot none ro 1,0 4077568 verity payload=PARTUUID=78B5F586-7B14-0A4B-B1F0-652FB69E08BE hashtree=PARTUUID=78B5F586-7B14-0A4B-B1F0-652FB69E08BE hashstart=4077568 al

## Evaluation

In [ ]:
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from hatedetection.metrics import compute_hate_metrics
from datasets import load_from_disk
import sys


"""
Todo esto es para evitar que la memoria salte por los aires por usar 'fork' en XLA
Ver: https://pytorch.org/xla/release/1.8/index.html#torch_xla.distributed.xla_multiprocessing.MpModelWrapper

"""


output_path = "/content/drive/Shareddrives/HateSpeech/models/bert-title-body-hate-speech-es/"

model = AutoModelForSequenceClassification.from_pretrained(
    output_path, return_dict=True
)
device = xm.xla_device()

model = model.to(device)
model.eval()
WRAPPED_MODEL = xmp.MpModelWrapper(model)


tokenizer = AutoTokenizer.from_pretrained(output_path)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    per_device_eval_batch_size=64,
)


trainer = Trainer(
    model=WRAPPED_MODEL,
    args=training_args,
    compute_metrics=compute_hate_metrics,
    
)

In [ ]:
test_dataset = load_from_disk("datasets/test")

In [ ]:
trainer.evaluate(test_dataset)

{'eval_accuracy': 0.9049634135590232,
 'eval_f1': 0.6705378973105135,
 'eval_loss': 0.8580359220504761,
 'eval_macro_f1': 0.8075054789941875,
 'eval_mem_cpu_alloc_delta': 583540,
 'eval_mem_cpu_peaked_delta': 794618,
 'eval_precision': 0.7437288135593221,
 'eval_recall': 0.6104618809126322,
 'eval_runtime': 971.1812,
 'eval_samples_per_second': 11.68,
 'init_mem_cpu_alloc_delta': 291614,
 'init_mem_cpu_peaked_delta': 15450}

In [ ]:
trainer.evaluate(test_dataset)

{'eval_accuracy': 0.9049634135590232,
 'eval_f1': 0.6705378973105135,
 'eval_loss': 0.8580359220504761,
 'eval_macro_f1': 0.8075054789941875,
 'eval_mem_cpu_alloc_delta': 314018,
 'eval_mem_cpu_peaked_delta': 907475,
 'eval_precision': 0.7437288135593221,
 'eval_recall': 0.6104618809126322,
 'eval_runtime': 257.4891,
 'eval_samples_per_second': 44.052}

## Resultados invirtiendo

In [ ]:
trainer.evaluate(test_dataset)

{'eval_accuracy': 0.9049634135590232,
 'eval_f1': 0.6705378973105135,
 'eval_loss': 0.8580359220504761,
 'eval_macro_f1': 0.8075054789941875,
 'eval_mem_cpu_alloc_delta': 274814,
 'eval_mem_cpu_peaked_delta': 909501,
 'eval_precision': 0.7437288135593221,
 'eval_recall': 0.6104618809126322,
 'eval_runtime': 264.3081,
 'eval_samples_per_second': 42.916}

## Results upside down

In [ ]:
trainer.evaluate(dev_dataset)

{'eval_accuracy': 0.9232374258730507,
 'eval_f1': 0.7383002620741295,
 'eval_loss': 0.6745567321777344,
 'eval_macro_f1': 0.8466612307089005,
 'eval_mem_cpu_alloc_delta': 371279,
 'eval_mem_cpu_peaked_delta': 666660,
 'eval_precision': 0.7679127725856698,
 'eval_recall': 0.7108868060562364,
 'eval_runtime': 827.3994,
 'eval_samples_per_second': 11.006,
 'init_mem_cpu_alloc_delta': 1202443,
 'init_mem_cpu_peaked_delta': 11001369}